In [1]:
import os, json, argparse, sys, time
import pandas as pd
from myEncoder.Encode import VitalSigns2Digital
import pkuseg#专业领域分词
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import numpy as np
np.seterr(divide='ignore',invalid='ignore')
from keras.utils import to_categorical

import tensorflow as tf
from tensorflow.compat.v1 import InteractiveSession
tf.keras.backend.clear_session()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from TransNet import TransNet
import pyttsx3
import tkinter as tk
from tkinter import ttk
import threading
os.environ["CUDA_VISIBLE_DEVICES"]="0" or "1"
tok = Tokenizer()
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

Using TensorFlow backend.


In [2]:
class TriageApp:
    def __init__(self, root, COLUMN, Tabel):
        self.root = root
        self.COLUMN = COLUMN
        self.Tabel = Tabel

        self.root.title("急诊分诊")

        # 创建输入框和标签
        self.entries = self.create_input_entries()
        # 创建按钮，点击后进行分诊
        triage_button = ttk.Button(root, text="分诊", command=self.triage, style='TButton', takefocus=False)
        triage_button.grid(row=0, column=2, columnspan=10, pady=5)

        # 创建显示结果的标签
        self.result_label = tk.Label(root, text="", font=("Helvetica", 14), fg="black", anchor='w', justify='left')
        self.result_label.grid(row=2, column=2, columnspan=2, pady=5)

        root.geometry("660x360")  # 设置画布窗口的尺寸
        root.resizable(0, 0)  # 设置画布窗口是否能变化大小，这里设置X和Y轴都不可改变

    def create_input_entry(self, row, column, label_text, default_value=""):
        label = tk.Label(self.root, text=label_text)
        label.grid(row=row, column=column, padx=10, pady=5, sticky=tk.W)
        entry = ttk.Entry(self.root, width=20)
        entry.grid(row=row, column=column + 1, padx=10, pady=10, sticky=tk.W)
        entry.insert(0, default_value)
        return entry

    def create_input_entries(self):
        labels = ["主诉", "入院方式", "体温（℃）", "心率 (次/min)", "呼吸 (次/min)", "血压 (mmHg)", "血氧饱和度 (%)"]
        defaults = [args.Chief_Complaint, args.Arrive, args.Temp, args.Pulse, args.Respiratory, args.BP, args.SpO2]
        entries = [self.create_input_entry(i, 0, label, default) for i, (label, default) in enumerate(zip(labels, defaults))]
        return entries

    def triage(self, **kwargs):
        values = [entry.get() for entry in self.entries]
        args.Chief_Complaint, args.Arrive, args.Temp, args.Pulse, args.Respiratory, args.BP, args.SpO2 = values
        Time = time.strftime('%Y/%m/%d %X', time.localtime())
        args.Triage_Time = Time
        VAR = [args.ID, args.Name, args.Sex, args.Birth,
               args.Triage_Time, args.Verify_Time, args.Chief_Complaint,
               args.Consumption, args.Retriage, args.Arrive, args.Temp,
               args.Pulse, args.Respiratory, args.BP, args.SpO2]
        VAR = ['空值' if str(i).isspace() or len(str(i)) == 0 else i for i in VAR]
        print("患者信息：",VAR[4:])

        # 在这里添加急诊分诊的逻辑，计算严重等级和就诊科室
        Level, Department, _ = self.detect_gui(VAR, model)

        INDEX = self.COLUMN
        if '级别' and '科室' not in INDEX:
            INDEX.extend(['级别', '科室'])
        DATA = _
        DATA.extend([Level, ', '.join(Department)])
        self.write(self.Tabel, DATA, INDEX)
        # 假设以下是简单的逻辑示例
        severity_level = Level  # 严重等级
        departments = Department  # 就诊科室（多个选项）

        # 在结果标签中显示结果
        self.result_label.config(text=f"严重等级：{severity_level}\n就诊科室：{', '.join(departments)}",
                                font=("Helvetica", 14, "bold"), fg="red")

        # 创建 OptionMenu 供用户选择最终就诊科室
        self.final_department_var = tk.StringVar(value=departments[0])  # 设置默认值
        option_menu = ttk.OptionMenu(self.root, self.final_department_var, *departments)
        option_menu.grid(row=2, column=10, columnspan=4, pady=10)

        # 使用多线程进行语音播报
        thread = threading.Thread(target=self.speak_result, args=(severity_level, departments))
        thread.start()

    def speak_result(self, severity_level, departments):
        # 使用 pyttsx3 进行语音播报
        engine = pyttsx3.init()
        engine.say(f"严重等级：{severity_level}。就诊科室：{', '.join(departments)}。")
        engine.runAndWait()

    def detect_gui(self, AVR, model):
        Time = time.strftime('%Y/%m/%d %H:%M:%S', time.localtime())
        start0 = time.perf_counter()

        VS, Data = self.data_input(self.COLUMN, AVR)

        WhitePath = args.WhiteListPath

        predictions = model.predict([VS, Data])
        y_predict1 = dicLevel[np.argmax(predictions[0])]
        y_predict2 = dicDepart[np.argmax(predictions[1])]
        end0 = time.perf_counter()
        args.Consumption = np.round((end0 - start0) * 1000, 3)
        variables = [args.ID, args.Name, args.Sex, args.Birth,
                     args.Triage_Time, args.Verify_Time,
                     args.Chief_Complaint, args.Consumption, args.Retriage,
                     args.Arrive, args.Temp, args.Pulse, args.Respiratory, args.BP, args.SpO2]

        RevisedDictionary = self.conditional_judgment(WhitePath, args.Chief_Complaint)
        Level, Department = self.results(RevisedDictionary, y_predict1, y_predict2)
        return Level, Department, variables

    def data_input(self, COLUMN, variables):
        data = pd.DataFrame(columns=COLUMN)
        for idx, i in zip(COLUMN, variables):
            data.at[0, idx] = i
        VS = [np.hstack(([to_categorical(self.func(data[i].values[0], i), LIB2[i]) for i in Parems]))]
        IM = self.unstructure(data)
        Data = np.hstack([VS, IM])
        return [VS], Data

    def unstructure(self, data):
        im = data['分诊印象'].apply(lambda x: self.segment_func(x, exclude))
        tok.fit_on_texts(im.values)
        with open('./JSON/word_index.json', 'r') as f:
            word_index = json.load(f)
        tok.word_index = word_index
        x = tok.texts_to_sequences(im.values)
        X = sequence.pad_sequences(x, maxlen=13)
        return X

    def segment_func(self, content, stopwords):
        segment = []
        segs = segcut.cut(content)
        for seg in segs:
            if seg not in stopwords:
                segment.append(seg)
        return segment

    def func(self, x, Name):
        funcs = LIB1[Name]
        result = getattr(VitalSigns2Digital(), funcs)(x)
        return result

    def conditional_judgment(self, WhitePath, Chief_Complaint):
        DepartWhiteList = os.listdir(WhitePath)
        Result = []
        for k in DepartWhiteList:
            Path = os.path.join(WhitePath, k)
            whitelist = pd.read_csv(Path, header=None)
            include = []
            for i in whitelist[0]:
                include.append(i)
            if any(word if word in Chief_Complaint else False for word in include):
                Result.append(k[:-4])
            else:
                pass
        return Result

#     def play_audio(self, ResultFinal):
#         # 播报
#         p = pyttsx3.init()
#         rate = p.getProperty('rate')
#         p.setProperty('rate', rate - 30)
#         p.say(ResultFinal)
#         p.runAndWait()

    def results(self, RevisedDictionary, y_predict1, y_predict2):
        if '入抢救室' in RevisedDictionary:
            y_predict1 = '一级'
            RevisedDictionary = ['入抢救室']
        elif len(RevisedDictionary) == 0:
            RevisedDictionary = [y_predict2]
        else:
            RevisedDictionary.append(y_predict2)
            RevisedDictionary = np.unique(RevisedDictionary)
        print("预测结果：%s-%s"%(y_predict1, ', '.join(RevisedDictionary)))
        return y_predict1, RevisedDictionary
    
    def write(self, data, variables, columns):
        for idx, i in zip(columns, variables):
            data.at[0, idx] = i
        data.to_csv(txt_path, mode='a', index=False, header=False)

In [3]:
if __name__ == '__main__':
    COLUMN = ['病人ID', '病人姓名', '性别', '出生日期', '分诊时间', '分诊修改时间', '分诊印象', '分诊耗时（ms）', '二次分诊', '到院方式', 'T℃', 'P(次/分)',
              'R(次/分)', 'BP(mmHg)', 'SpO2']
    Time = time.strftime('%Y/%m/%d %X', time.localtime())

    parser = argparse.ArgumentParser()
    parser.add_argument('--ID', type=str, default='001xxx')
    parser.add_argument('--Name', type=str, help='Name of patient', default='小明')
    parser.add_argument('--Sex', type=str, help='[男, 女]', default='无')
    parser.add_argument('--Birth', type=str, help='2000/1/8', default='2000/1/8')
    parser.add_argument('--Triage_Time', type=str, help='2023/11/07 21:54', default=Time)
    parser.add_argument('--Verify_Time', type=str, default=Time, help='修正时间')
    parser.add_argument('--Consumption', type=str, default='空值', help='耗时')
    parser.add_argument('--Chief_Complaint', type=str, default='空值', help='主诉')
    parser.add_argument('--Retriage', type=str, help='是， 否', default='否')
    parser.add_argument('--Arrive', type=str, default='步入')
    parser.add_argument('--Temp', type=float, default=36.3)
    parser.add_argument('--Pulse', type=float, default=80)
    parser.add_argument('--Respiratory', type=int, default=18, help='呼吸频率')
    parser.add_argument('--BP', type=str, help='90-140/60-90', default='100/80')
    parser.add_argument('--SpO2', type=int, help='血氧饱和度：95-100', default='98')
    parser.add_argument('--weight', help='权重文件', default='./weight/TransNet.h5')
    parser.add_argument('--WhiteListPath', help='白名单地址', default=r'./StopWords/whitelist')

    args = parser.parse_args(args=[])
    Parems = ['到院方式', '性别', '出生日期', 'T℃', 'P(次/分)', 'R(次/分)', 'BP(mmHg)', 'SpO2']
    Funcs = ['Arr_way', 'Gender', 'Age', 'Temperature', 'Pulse', 'Respiration', 'BloodPressure', 'SpO2']
    dicLevel = {0: '一级', 1: '二级', 2: '三级', 3: '四级'}
    dicDepart = {0: '产科', 1: '内科', 2: '创伤救治中心', 3: '外科', 4: '妇科', 5: '眼科', 6: '神经内科', 7: '神经外科', 8: '耳鼻喉', 9: '骨科'}

    LIB1 = {idx: i for idx, i in zip(Parems, Funcs)}
    Length = [4, 3, 5, 3, 3, 3, 6, 3]
    LIB2 = {idx: i for idx, i in zip(Parems, Length)}
    model = TransNet()
    model.load_weights(args.weight)

    def creat_csv(file_path, COLUMN):
        if not os.path.exists(file_path):
            tabel = pd.DataFrame(columns=COLUMN)
            tabel.to_csv(file_path, index=False, header=True, encoding="utf_8_sig")
        else:
            tabel = pd.read_csv(file_path, engine='python')
        return tabel

    stopwords = pd.read_csv("./StopWords/common.txt",quoting=3, sep="\t", encoding='utf-8')
    exclude = []
    for i in np.asarray(stopwords):
        exclude.append(i[0])
    exclude.extend(' ')
    segcut = pkuseg.pkuseg(model_name = "medicine", user_dict = "default", postag = False)  # 程序会自动下载所对应的细领域模型

    txt_path = r"./result/Result.csv"
    tabel = creat_csv(txt_path, COLUMN)
    # 创建主窗口
    root = tk.Tk()
    app = TriageApp(root, COLUMN, tabel)
    style = ttk.Style()
    style.configure("TButton", padding=(10, 5))
    # 启动主循环
    root.mainloop()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
患者信息： ['2023/11/27 16:01:33', '2023/11/27 16:01:21', '脑卒中', '空值', '否', '步入', '36.3', '80', '18', '100/80', '98']
预测结果：四级-神经内科
患者信息： ['2023/11/27 16:01:42', '2023/11/27 16:01:21', '脑卒中', 1021.646, '否', '步入', '36.3', '80', '18', '100/80', '94']
预测结果：三级-神经内科
患者信息： ['2023/11/27 16:01:57', '2023/11/27 16:01:21', '动物致伤', 19.154, '否', '步入', '36.3', '80', '18', '100/80', '96']
预测结果：四级-外科
患者信息： ['2023/11/27 16:03:09', '2023/11/27 16:01:21', '孕38+4   宫缩 ', 25.437, '否', '步入', '36.3', '80', '18', '100/80', '96']
预测结果：四级-产科
患者信息： ['2023/11/27 16:03:18', '2023/11/27 16:01:21', '头晕   恶心', 28.666, '否', '步入', '36.3', '80', '18', '100/80', '96']
预测结果：四级-内科
患者信息： ['2023/11/27 16:04:28', '2023/11/27 16:01:21', '凝血功能异常', 21.623, '否', '120', '36.5', '114', '空值', '116/64', '100']
预测结果：三级-内科
患者信息： ['2023/11/27 16: